<a href="https://colab.research.google.com/github/shivam-karthik/sentiment_analysis/blob/main/sentiment_analysis_skar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing necessary libraries

import nltk, re, string
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pickle  #load and store modal and data from the disk

In [2]:
# creation of function for preprocessing of the tweets(the data)
 
def process_tweet(tweet):
   #using porterstemmer algorithm for cleaning words
    stemmer = nltk.PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = nltk.TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and
                word not in string.punctuation):
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


  
  

In [3]:
#creating function to count frequency of occurance of words in  a tweet
#this function will return a dictionary of keyname equivalemt of a tuple and a-
# value of either 0 or 1
def build_freqs(tweets, ys):
     """Build frequencies 
     i/p 
        tweets : list of tweet
        ys : m*1 array with sentiment label(0 or 1) of each tweet

        o/p- freq- a dictionary mapping each (word,sentiment) pair to its 
        frequency
     """

    #removing axes of length one from ys using squeeze() to make it zero dimensional array.
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.

     yslist = np.squeeze(ys).tolist()

   

    #creating a zipped list of yslist, tweets which have tuple elements ('y', tweet)
    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
     freqs = {}
     for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

     return freqs
      

In [6]:
#checking how above code works

tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0 ,0, 0, 0]
res= build_freqs(tweets, ys)
print(res)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}


In [5]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# selecting the dataset
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')


In [8]:
# spliting tehe dataset into training and test set
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]



In [9]:
train_x = train_pos + train_neg
test_x = test_pos + test_neg



In [10]:
#combine positive and negative labels
#we are building our y -target varaible here
#numpy.append appends both trains pos and train neg along one list or axis
#we have passed dimensions of array in numpy.append##
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [11]:
# create frequency dictionary

freqs = build_freqs(train_x, train_y)

In [12]:
#check out the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11346


In [13]:
#test the function below

print("this is an example of a positive tweet: \n", train_x[22])
print("\nthis is an example of the processed version of the tweet", process_tweet(train_x[22]))

this is an example of a positive tweet: 
 @gculloty87 Yeah I suppose she was lol! Chat in a bit just off out x :))

this is an example of the processed version of the tweet ['yeah', 'suppos', 'lol', 'chat', 'bit', 'x', ':)']


In [14]:
# we will build a logiatic regression model from scratch

#lets create a sigmoid function
def sigmoid(z):
    """
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    """
    zz = np.negative(z)
    h = 1 / (1 + np.exp(zz))
    return h

In [15]:
#lets define the cost function and Gradientfor our lofistic model
def gradientDescent(x, y, theta, alpha, num_iters):

    """
    Input:
        x: matrix of features which is (m,n+1)
        y:corresponding labels of the input matrix x, dimensions (m,1)
        theta : weight vector of dimension (n+1,1)
        alpha: learning vector
        num_iters : number of iterations you want to train your model for
        m: the number of rows in matrix x
    Output:
         J: the final cost
         theta: final weight vector
        
    """  
    #cost function
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    for i in range(0, num_iters):
        z = np.dot(x, theta)
        h = sigmoid(z)
        # calculate the cost function
        cost = -1. / m * (np.dot(y.transpose(), np.log(h)) + np.dot((1 - y).transpose(), np.log(1 - h)))
        # update the weights theta
        theta = theta - (alpha / m) * np.dot(x.transpose(), (h - y))

    cost = float(cost)
    return cost, theta


    
    

In [16]:
#feature extraction

def extract_features(tweet, freqs):
  """
  Input:
       tweeet: a list of words for one tweet
       freqs : a dictionary corresponding to the frequencies of each tuple(word,label)
  output:
       x: A feature vector of dimension (1,3)
  """

  word_l = process_tweet(tweet)
  x= np.zeros((1,3))

  #bias term is set to 1
  x[0,0] = 1

  for word in word_l:
    #increment the word count for the positive label 1
    #using dictionary.get(keyname, value to be returned if key is not found)
    
    x[0,1] += freqs.get((word, 1.0),0)
    #increment the word count for the negative label 0
    x[0,2] += freqs.get((word, 0.0), 0)

  assert (x.shape == (1,3))
  return x


In [17]:
# #just a way to go about things happening here or above#
# word_l = process_tweet(train_x[22])
# for word in word_1:
#   print(freqs.get((word, 0.0), 0))

In [18]:
# test on training data
tmp1 = extract_features(train_x[22], freqs)
print(tmp1)

[[1.000e+00 3.006e+03 1.240e+02]]


In [ ]:
#above three numbers are our feautre vector
#hypothesis h(x) = b1+ b2*x1 + b3*x2 , b2,b3 will be calculated by cost and gradient 
#as bias = b1 =1 also x1 and x2 are positive and negative feature set


In [19]:
# Training Your Model

# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :] = extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
# these values are predefined (Andrew NG)
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)


In [20]:
def predict_tweet(tweet, freqs, theta):
    """
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    """
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    y_pred = sigmoid(np.dot(x, theta))

    return y_pred

In [21]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input:
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    # the list for storing predictions
    y_hat = []

    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            y_hat.append(0)

    accuracy = (y_hat == np.squeeze(test_y)).sum() / len(test_x)

    return accuracy

In [22]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"LR model's accuracy = {tmp_accuracy:.4f}")

LR model's accuracy = 0.9950


In [23]:
#predict your tweet

def pre(sentence):
  yhat = predict_tweet(sentence, freqs, theta)
  if yhat> 0.5:
     return 'positive sentiment'
  elif yhat == 0:
    return 'neutral sentiment'
  else: 
    return 'negative sentiment'


In [ ]:
test_tweet = 'find your fire'

res = pre(test_tweet)
print(res)

negative sentiment
